In [1]:
import json
from pprint import pprint
import requests  # requests library will send and receive data from the escalate server
from requests.api import post
import pandas as pd

In [2]:
base_url = 'http://localhost:8000/api'  # local dev server
login_data = {
    'username': 'mtynes',
    'password': 'hello1world2'
}

r_login = requests.post(f'{base_url}/login', data=login_data)
token = r_login.json()['token']
token_header = {'Authorization': f'Token {token}'}
content_type_header = {'content-type': 'application/json'} # for most 

In [3]:
def patch_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """PATCH `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.patch(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        #print('POST: OK, returning new resource dict')
        return r.json()
    #print('POST: FAILED, returning response object')
    return r

def put_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """PATCH `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.put(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        #print('POST: OK, returning new resource dict')
        return r.json()
    #print('POST: FAILED, returning response object')
    return r


def post_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """POST `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.post(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    #print(r)
    if r.ok: 
        #print('POST: OK, returning new resource dict')
        return r.json()
    else:
        print(f'POST: FAILED, returning response object {r.reason}')
        print(r.json())

    return r


def get_data(endpoint, data={}, headers={**token_header}):
    """Make GET request with `data` to `endpoint` in ESCALATE API using `headers`
    
    return: (dict|list|requests.Response), bool
    """
    r = requests.get(f'{base_url}/{endpoint}/', params=data, headers=headers)
    print(r)
    if r.ok: 
        print('GET: OK')
        
        resp_json = r.json()        
        
        # handle cases: one vs many results
        if resp_json.get('count') is None: # edge case: template edit
            return r.json()
        elif resp_json.get('count') >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
        else:
            print('GET: FAILED, returning response object')
    return r

In [4]:
val_type = {'num': get_data('type-def', {'category':'data', 'description': 'num'})[0],
    'text': get_data('type-def', {'category':'data','description': 'text'})[0],
            }

<Response [200]>
GET: OK
Found 1 resources, returning list of dicts)
<Response [200]>
GET: OK
Found 1 resources, returning list of dicts)


In [5]:
prop_def_cols = ['description', 'short_description', 'val_type', 'val_unit', 'property_def_class']

prop_def_data = [
('plate well location', 'well_loc', val_type['text']['url'], None, 'intrinsic'), 
('plate well volume', 'well_vol', val_type['text']['url'], None, 'intrinsic'), 
('plate well ord', 'well_ord', val_type['num']['url'], None, 'intrinsic'),('molecular-weight', 'mw', val_type['text']['url'], 'g/mol', 'intrinsic'),
('concentration_molarity', 'cm', val_type['num']['url'], 'M', 'extrinsic'),
]
property_defs= {}

for values in prop_def_data:
    data = dict(zip(prop_def_cols, values))
    property_defs[data['short_description']] = post_data('property-def', data=data)


In [6]:
well_plate_96 = post_data('material', {'description': 'Plate: 96 Well', 'material_class': 'template', 'consumable': False})


In [7]:
well_plate_96

{'url': 'http://localhost:8000/api/material/e345e274-9f09-4e35-874c-6d8cfa3a5aa8/',
 'uuid': 'e345e274-9f09-4e35-874c-6d8cfa3a5aa8',
 'edocs': [],
 'tags': [],
 'notes': [],
 'property': [],
 'add_date': '2021-06-07T23:23:38.814772Z',
 'mod_date': '2021-06-07T23:23:38.814805Z',
 'description': 'Plate: 96 Well',
 'consumable': False,
 'material_class': 'template',
 'status': None,
 'actor': None,
 'identifier': [],
 'material_type': []}

In [8]:
# Create 96 well plate
import itertools
well_letters = list('ABCDEFGH')
well_numbers = [1, 2, 3, 4, 5, 6, 7, 8 , 9, 10, 11, 12]
wells = list(itertools.product(well_letters, well_numbers))
wells = [f'{w[0]}{w[1]}' for w in wells]
well_rest_data = {}
for ord, well in enumerate(wells):
    well_rest_data[well] = post_data('material', {'description': f'96 Well Plate well#: {well}', 'material_class': 'template', 'consumable': False})
    post_data('composite-material', {'composite': well_plate_96['url'], 'component': well_rest_data[well]['url'], 'addressable': True})
    post_data(f'material/{well_rest_data[well]["uuid"]}/property', {
            "property_val": {
                "value": str(ord),
                "type": "text",
                "unit": ""
            },
            "property_def": property_defs['well_ord']['url'],
            "property_class": "nominal"
        })
    post_data(f'material/{well_rest_data[well]["uuid"]}/property', {
            "property_val": {
                "value": well,
                "type": "text",
                "unit": ""
            },
            "property_def": property_defs['well_loc']['url'],
            "property_class": "nominal"
        })
    post_data(f'material/{well_rest_data[well]["uuid"]}/property', {
            "property_val": {
                "value": [0.5, 10],
                "type": "array_num",
                "unit": "mL"
            },
            "property_def": property_defs['well_vol']['url'],
            "property_class": "nominal"
        })

In [9]:
# Material types
material_types_rest_data = {}
types = ['stock solution', 'human prepared', 'solute', 'solvent']
for type in types:
    material_types_rest_data[type] = post_data('material-type', data={'description': type})

In [10]:
# Materials
hc_inventory = post_data('inventory', {'description': 'HC Test Inventory'})
material_cols = ['description', 'consumable', 'material_class', 'material_type']
material_data = [('Water', False, 'template', []),
                ('Gamma-Butyrolactone', False, 'template', []),
                ('Formic Acid', False, 'template', []),
                ('Lead Diiodide', False, 'template', []),
                ('Ethylammonium Iodide', False, 'template', []),
                ('Stock A', True, 'model', [material_types_rest_data['stock solution']['url'], material_types_rest_data['human prepared']['url']]),
                ('Stock B', True, 'model', [material_types_rest_data['stock solution']['url'], material_types_rest_data['human prepared']['url']]),
                ('Stock FAH', True, 'model'),]
material_rest_data = {}

for mat_data in material_data:
    data = dict(zip(material_cols, mat_data))
    material_rest_data[data['description']] = post_data('material', data=data)



In [11]:
# Composite Material
material_composite_columns = ['composite', 'component', 'addressable', 'material_type']
solvent = material_types_rest_data['solvent']['url']
solute = material_types_rest_data['solute']['url']


material_composite_data = [
    ('Stock A', 'Lead Diiodide', False, [solute]), 
    ('Stock A', 'Ethylammonium Iodide', False, [solute]), 
    ('Stock A', 'Gamma-Butyrolactone', False, [solvent]), 
    ('Stock B', 'Ethylammonium Iodide', False, [solute]), 
    ('Stock B', 'Gamma-Butyrolactone', False, [solvent]),
    ('Stock FAH', 'Formic Acid', False, []),
    ('Stock FAH', 'Water', False, [])]


material_composite_rest_data = {}

for mat_data in material_composite_data:
    composite_name, component_name, addressable, material_type = mat_data 
    composite_url = material_rest_data[composite_name]['url']
    component_url = material_rest_data[component_name]['url']
    data = dict(zip(material_composite_columns, (composite_url, component_url, True, material_type)))
    material_composite_rest_data[(composite_name, component_name)] = post_data('composite-material', data=data)

In [12]:
# Adding properties to materials
material_prop_data = [(('Stock A', 'Lead Diiodide'), 1.1),
    (('Stock A', 'Ethylammonium Iodide'), 2.2),
    (('Stock B', 'Ethylammonium Iodide'), 3.99),
    (('Stock FAH', 'Formic Acid'), 23.6),]

for composite_key, value in material_prop_data:
    uuid = material_composite_rest_data[composite_key]['uuid']
    post_data(f'composite-material/{uuid}/property', {
                "property_val": {
                    "value": value,
                    "type": "num",
                    "unit": "M"
                },
                "property_def": property_defs['cm']['url'],
                "property_class": "nominal"
            })


In [13]:
# Adding materials to inventory
inventory_mat_cols = ["description", "inventory", "material"]
inventorty_mat_data = [('Wf1 Plate', hc_inventory['url'], well_plate_96['url']),
('Stock A', hc_inventory['url'], material_rest_data['Stock A']['url']), 
('Stock B', hc_inventory['url'], material_rest_data['Stock B']['url']), 
('Stock FAH', hc_inventory['url'], material_rest_data['Stock FAH']['url']), 
('Neat GBL', hc_inventory['url'], material_rest_data['Gamma-Butyrolactone']['url'])]

inventory_material_rest_data = {}

for im_data in inventorty_mat_data:
    data = dict(zip(inventory_mat_cols, im_data))
    inventory_material_rest_data[data['description']] = post_data('inventory-material', data=data)


In [14]:
# Create experiment template
wf1_experiment = post_data('experiment-template', {
    "ref_uid": "test_wf_1",
    "description": "test_wf_1",
})

In [15]:
# Add bill of materials
bom = post_data('bill-of-materials', data={'experiment': wf1_experiment["url"], 'description': 'Test WF1 Materials'})


In [16]:
# Add materials to bom
bom_materials_cols = ['description', 'inventory_material']
bom_materials_data = [('Acid', inventory_material_rest_data['Stock FAH']['url']),
('Solvent', inventory_material_rest_data['Neat GBL']['url']),
('Stock A', inventory_material_rest_data['Stock A']['url']),
('Stock B', inventory_material_rest_data['Stock B']['url']),
('Plate', inventory_material_rest_data['Wf1 Plate']['url']),]

bom_materials_rest_data = {}

for bom_mat in bom_materials_data:
    data = dict(zip(bom_materials_cols, bom_mat))
    data['bom'] = bom['url']
    bom_materials_rest_data[data['description']] = post_data('bom-material', data=data)

In [17]:
# Adding workflow types
workflow_types_cols = ['description']
workflow_types_data = [['template']]

workflow_types_rest_data = {}

for wf_type in workflow_types_data:
    data = dict(zip(workflow_types_cols, wf_type))
    print(data)
    workflow_types_rest_data[data['description']] = post_data('workflow-type', data)


{'description': 'template'}


In [18]:
# Adding to workflow
workflow_cols = ['workflow_type', 'description']
template = workflow_types_rest_data['template']['url']
workflow_data = [(template, 'Preheat Plate'),
(template, 'Dispense Solvent'),
(template, 'Dispense Stock A'),
(template, 'Dispense Stock B'),
(template, 'Dispense Acid Vol 1'),
(template, 'Heat Stir 1'),
(template, 'Dispense Acid Vol 2'),
(template, 'Heat Stir 2'),
(template, 'Heat'),]

workflow_rest_data = {}

for wf_data in workflow_data:
    data = dict(zip(workflow_cols, wf_data))
    #print(data)
    workflow_rest_data[data['description']] = post_data('workflow', data)

In [19]:
# Adding to experiment workflow

exp_wf_cols = ["experiment_workflow_seq", "experiment", "workflow"]
exp_wf_data = [
    (1, wf1_experiment['url'], workflow_rest_data['Dispense Solvent']["url"]),
    (2, wf1_experiment['url'], workflow_rest_data['Dispense Stock A']["url"]),
    (3, wf1_experiment['url'], workflow_rest_data['Dispense Stock B']["url"]),
    (4, wf1_experiment['url'], workflow_rest_data['Dispense Acid Vol 1']["url"]),
    (5, wf1_experiment['url'], workflow_rest_data['Dispense Acid Vol 2']["url"]),
]

for exp_wf in exp_wf_data:
    data = dict(zip(exp_wf_cols, exp_wf))
    post_data('experiment-workflow', data=data)

In [20]:
param_def_cols = ['description', 'default_val', 'required', 'unit_type']
param_def_data = [
    ('volume', {'value':0, 'type':'num', 'unit':'mL'}, True, 'mL'),
    ('heat', {'value':0, 'type':'num', 'unit':'degC'}, True, 'degC')]
param_def_rest_data = {}
for param_def in param_def_data:
    data = dict(zip(param_def_cols, param_def))
    param_def_rest_data[data['description']] = post_data('parameter-def', data)

In [21]:
action_def_cols = ['description', 'parameter_def']
action_def_data = [('dispense',[param_def_rest_data['volume']['url']] ), ('heat', [param_def_rest_data['heat']['url']]), ('heat_stir', [param_def_rest_data['heat']['url']])]
action_def_rest_data = {}
for action_def in action_def_data:
    data = dict(zip(action_def_cols, action_def))
    print(data)
    action_def_rest_data[data['description']] = post_data('action-def', data)

{'description': 'dispense', 'parameter_def': ['http://localhost:8000/api/parameter-def/59099637-8a18-45dd-898d-59b16f04ee83/']}
{'description': 'heat', 'parameter_def': ['http://localhost:8000/api/parameter-def/ce3ef44c-89b4-44af-b5b7-5de374ca9c83/']}
{'description': 'heat_stir', 'parameter_def': ['http://localhost:8000/api/parameter-def/ce3ef44c-89b4-44af-b5b7-5de374ca9c83/']}


In [22]:
# Adding workflow action sets
action_set_cols = ["description", "workflow", "action_def",   ]
action_set_data = [
    ('Dispense Solvent', workflow_rest_data['Dispense Solvent']['url'], action_def_rest_data['dispense']['url']),
    ('Dispense Stock A', workflow_rest_data['Dispense Stock A']['url'], action_def_rest_data['dispense']['url']),
    ('Dispense Stock B', workflow_rest_data['Dispense Stock B']['url'], action_def_rest_data['dispense']['url']),
    ('Dispense Acid Vol 1', workflow_rest_data['Dispense Solvent']['url'], action_def_rest_data['dispense']['url']),
    ('Dispense Acid Vol 2', workflow_rest_data['Dispense Solvent']['url'], action_def_rest_data['dispense']['url']),]

action_set_rest_data = {}
for action_set in action_set_data:
    data = dict(zip(action_set_cols, action_set))
    action_set_rest_data[data['description']] = post_data('action', data)

{'start_date': ['This field is required.'],
 'end_date': ['This field is required.']}